In [3]:
pal = [(39/255,95/255,153/255),(80/255,176/255,220/255),
       (237/255,244/255,248/255),
       (146/255,144/255,142/255),
       (78/255,78/255,80/255),
(235/255,154/255,88/255),(87/255,126/255,130/255)]

from dask import compute, delayed
from dask.distributed import Client

import sys
import math
sys.path.append("/Users/rcarlson/Documents/GitHub/lasagna3/snakes")
from ops.imports_ipython import *

import javabridge
import bioformats
from scipy import ndimage
import matplotlib.pyplot as plt
import glob

/Users/rcarlson/miniconda3/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## save IMS to tif

In [ ]:
javabridge.start_vm(class_path=bioformats.JARS)


loc = '/Volumes/BeccaDrive/M152/*.ims'
files = np.sort(glob.glob(loc))

for f in files[-7:]:
    well = f.split('final-')[1].split('-')[0]
    site = f.split('.ims')[0].split('_F')[-1]
    
    
    istart = 5
    iend = 19

    img = []
    for i in range(istart,iend):
        with bioformats.ImageReader(f) as reader:
            img.append(reader.read(z=i))
            reader.close()

    ## use mito channel to find in-focus z

    z = []
    for i in range(len(img)):
        z.append(ndimage.laplace(img[i][:,:,2]).var())
    bestz = z.index(max(z))

    print(well,' ',site)
    print('max lap var: ', max(z))
    print('best z found: ', bestz + istart)

    # redefine image to best z only
    with bioformats.ImageReader(f) as reader:
            img = (reader.read(z=bestz + istart))
            reader.close()
            
    # save image
    img = np.moveaxis(img,-1,0)
    save(str('/Volumes/Seagate Backup Plus Drive/M152/tifs/Well' + well + '_Site' + site + '.tif'), img)

## Plot data

In [4]:
from pandas.errors import EmptyDataError


# load pheno dfs
@delayed
def read_csv_pheno(f):
    try:
        df = pd.read_csv(f)
                      
    except EmptyDataError:
        df = pd.DataFrame()
        print('empty   ', f.split('/')[-1])
    return df

loc = '/Users/rcarlson/Documents/GitHub/Zenodo_IRF3/Figure5/BJ1/tifs/process/*.mavs.csv'

#loc = '/Volumes/BeccaDrive/M152/tifs/process/*.mavs.csv'
files = np.sort(glob.glob(loc))
print(len(files))

from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_csv_pheno, files), scheduler='threads'))
    


160
[#######################                 ] | 57% Completed |  0.5sempty    WellB02_site-20.mavs.csv
[########################################] | 100% Completed |  0.9s


In [5]:
df['gene'] = 'nontargeting'
cond = df.well.isin(['B01','B02','C01','C02'])
df.loc[cond, 'gene'] = 'ATP13A1'

In [7]:
feature = 'channel_corrch1_cell_corr'
scipy.stats.ttest_ind(df[df.gene == "nontargeting"][feature], df[df.gene == "ATP13A1"][feature])

Ttest_indResult(statistic=33.39935792633864, pvalue=1.7716040619197044e-205)

In [ ]:
sns.set(font_scale = 1.2, style = 'white')

sns.swarmplot(data = df.sort_values('gene',ascending=False), 
              x= 'gene', y = 'channel_corrch1_cell_corr', color = pal[2])

sns.despine(top = True, right = True)
plt.xlabel('')
plt.ylabel('')
plt.show()
##